## 1.Transcript Selection

In [ ]:
import pandas as pd
import numpy as np

### 1.1 Transcript Summary
* The Top10Gene_exon.csv is generated by Jiaqi in Gene Prioritization Process.
* The V2.x and V3.x and transcript_start is used for locating SNP.
* The V6.x is the wt nucleotide and the V7.x is the SNP

In [ ]:
data=pd.read_csv("Top10genes_exon.csv")
data["count"]=[1]*data.shape[0]
data.head()

,V1.x,V2.x,V3.x,V6.x,V7.x,gene_id,transcript_id,transcript_start,transcript_end,count
0,chr20,5280261,5280262,G,T,ENSG00000101292.8_9,ENST00000217270.4_5,5279906,5297060,1
1,chr20,5281326,5281327,G,A,ENSG00000101292.8_9,ENST00000217270.4_5,5279906,5297060,1
2,chr20,5281252,5281253,G,A,ENSG00000101292.8_9,ENST00000217270.4_5,5279906,5297060,1
3,chr20,5280332,5280333,T,C,ENSG00000101292.8_9,ENST00000217270.4_5,5279906,5297060,1
4,chr20,5281412,5281413,C,T,ENSG00000101292.8_9,ENST00000217270.4_5,5279906,5297060,1


In [ ]:
summary=pd.pivot_table(data, values='count', index=['gene_id','transcript_id'],aggfunc=np.sum)
summary.sort_values(by=["gene_id",'count',], ascending=False)

count
gene_id               transcript_id              
ENSG00000196090.13_14 ENST00000356100.6_8      14
ENSG00000149633.12_15 ENST00000279024.9_10     14
ENSG00000149596.7_7   ENST00000372980.4_6      25
ENSG00000149488.14_7  ENST00000358864.2_3      17
ENSG00000130702.15_9  ENST00000252999.7_5      30
ENSG00000130589.16_9  ENST00000467148.2_7      26
                      ENST00000427522.6_4      22
ENSG00000101292.8_9   ENST00000217270.4_5      18
                      ENST00000678059.1_2      18
                      ENST00000678254.1_4      18
ENSG00000101276.18_16 ENST00000675066.1_2      15
ENSG00000101181.18_9  ENST00000370823.8_3      15
ENSG00000101160.15_9  ENST00000681457.1_1      18

In [ ]:
#summary.to_csv("summary.csv")

### 1.2 Transcript selection based on PDB data
* The Protein.csv is manually created by Keyi 
* The first three column is the accession id, the column pdb and alphafold contains the information about whether the sturcture information is adopted by Ensembl. The protein_length_match column contains information about whether the downloaded structure has the same length as the wt sequence (in later part, it is furthur verified to see if the sequence of the structure matches with the wt amino acid sequence

In [ ]:
protein=pd.read_csv("Protein.csv")
protein

,gene_id,transcript_id,uniprot,pdb,alphafold,mutation,protein_length,Name,Unnamed: 8,protein_length_match
0,ENSG00000101160.15_9,ENST00000681457.1_1,no,Retained intron,no,18,0,NaN,1,no
1,ENSG00000101181.18_9,ENST00000370823.8_3,Q9H4K7-1,no,yes,15,406,MTG2-201,2,yes
2,ENSG00000101276.18_16,ENST00000675066.1_2,A0A6Q8PFQ2,no,no,15,364,SLC52A3-207,3,yes
3,ENSG00000101292.8_9,ENST00000678059.1_2,A0A7I2V3D2,no,no,18,348,PROKR2-202,4,yes
4,ENSG00000130589.16_9,ENST00000427522.6_4,Q9BYK8-2,no,no,22,2080,HELZ2-202,5,no
5,ENSG00000130702.15_9,ENST00000252999.7_5,O15230-1,no,no,30,3695,LAMA5-201,6,no
6,ENSG00000149488.14_7,ENST00000358864.2_3,Q8TDI7-1,no,yes,17,906,TMC2-201,7,no
7,ENSG00000149596.7_7,ENST00000372980.4_6,Q9BR39-1,no,yes,25,696,JPH2-202,8,yes
8,ENSG00000149633.12_15,ENST00000279024.9_10,Q5JYT7,no,yes,14,1200,KIAA1755-201,9,yes
9,ENSG00000196090.13_14,ENST00000356100.6_8,B1AJR8,no,no,14,1450,PTPRT-201,10,no


## 2. Data Preprocessing

### 2.1 Read in all sequence
* Each transcript has a folder named by its corresponding index. In each folder, there are three fa files downloaded from Ensembl(https://useast.ensembl.org/index.html) and a pdb.fa file downloaded from the UniProt. genomics.fa is downloaded at the transcript page, exon panel by selecting download genomic sequence with 0 flanking sequence. exon.fa is downloaded at the transcript page, cDNA panel by selecing only exon. The amino.fa is downloaded at the transcript page, protein panel. The table.csv contains information about exon and intron and is downloaded at the transcript page and exon panel.
* The structure in each folder is downloaded from alphafold database (https://alphafold.ebi.ac.uk/) using accession number. Only structure with exactly the same sequence as WT are downloaded and analyzed

In [ ]:
import os
import string
folders=[]
for folder in os.listdir("data/"):
    if "." not in folder:
        folder=folder+"/"
        folders.append(folder)
folders

['9/', '8/', '4/', '3/', '2/']

In [ ]:
# save all sequences
all_sequence={}
for folder in folders:
    all_sequence[folder[:-1]]={}
    for file in os.listdir("data/"+folder):
        if "fa" in file:
            f=open("data/"+folder+file)
            ls=[]
            for line in f:
                if not line.startswith('>'):    
                    ls.append(line.replace('\n',''))
            sequence="".join(ls)
            all_sequence[folder[:-1]][file.split(".")[0]]=sequence
            f.close()

In [ ]:
# save corresponding transcrip_id
transcript_ids = {}
for item in all_sequence.keys():
    index=int(item)-1
    transcript_ids[item]=protein["transcript_id"].to_list()[index]
transcript_ids

{'9': 'ENST00000279024.9_10',
 '8': 'ENST00000372980.4_6',
 '4': 'ENST00000678059.1_2',
 '3': 'ENST00000675066.1_2',
 '2': 'ENST00000370823.8_3'}

### 2.2 Check PDB sequence and Ensembl Amino sequence

In [ ]:
all_sequence.keys()

dict_keys(['9', '8', '4', '3', '2'])

In [ ]:
all_sequence["9"].keys()

dict_keys(['genomic', 'amino', 'pdb', 'exon'])

In [ ]:
# check if the wt sequence from the ensemble matches the sequence of the corresponding structure in alphafold database
for item in all_sequence.keys():
    if all_sequence[item]["pdb"]==all_sequence[item]["amino"]:
        print("True: "+item)

True: 9
True: 8
True: 4
True: 3
True: 2


### 2.3 Create Mutant Sequence and Save

In [ ]:
# table for splicing info
def table_info(filename):
    data=pd.read_csv(filename,thousands=',').iloc[1:-1,:]
    data[["Start","End","Length"]]=data[["Start","End","Length"]].astype("int")
    data=data[data["Exon / Intron"].apply(lambda x: x.startswith("E"))]
    return data

In [ ]:
# Is it anti-sense
def anti(table):
    if (table["Start"].to_list()[0])>(table["End"].to_list()[0]):
        return True
    else:
        return False

In [ ]:
# complimentary sequence
def compliment(sequence):
    comp_dict = {
        "A":"T",
        "T":"A",
        "G":"C",
        "C":"G",
    }
    sequence_list = list(sequence)
    sequence_list = [comp_dict[base] for base in sequence_list]
    string = ''.join(sequence_list)
    return string

In [ ]:
# Create Mutant: This function can introduce mutation in to the right place through relative position.
# It can be sued for both sense and anti-sense cases
def mutants(transcript_table,id,wt_seq,anti):
    transcript_table=transcript_table[transcript_table["transcript_id"].apply(lambda x: x.startswith(id))]
    if anti:
        wt_seq=compliment(wt_seq)
        wt_seq=list(wt_seq.strip())
        loc=(transcript_table["V3.x"]-transcript_table["transcript_start"]).to_list()
        n=0
        for i in range(len(loc)):
            item=loc[i]
            if (wt_seq[-item]==transcript_table["V6.x"].to_list()[i])==False:
                n+=1
            wt_seq[-item]=transcript_table["V7.x"].to_list()[i] 
        print("Mismatch between reference and ensembl: "+str(n))
        wt_seq="".join(wt_seq)
        wt_seq=compliment(wt_seq)
    else:
        loc=(transcript_table["V2.x"]-transcript_table["transcript_start"]).to_list()
        wt_seq=list(wt_seq.strip())
        n=0
        for i in range(len(loc)):
            item=loc[i]
            if (wt_seq[item]==transcript_table["V6.x"].to_list()[i])==False:
                n+=1
           # else:
             #   wt_seq[item]=transcript_table["V7.x"].to_list()[i]
        print("Mismatch between reference and ensembl: "+str(n))
        wt_seq="".join(wt_seq)
    return wt_seq

In [ ]:
# splice: exract exons from the given transcript sequence
def exons(table,wt_seq):
    relative_start=[]
    start=table["Start"].to_list()[0]
    for item in table["Start"].to_list():
        relative_start.append(np.abs(item-start))
    relative_end=[]
    for item in table["End"].to_list():
        relative_end.append(np.abs(item-start)+1)
    final_seq=""
    for item in zip(relative_start,relative_end):
        final_seq+=wt_seq[item[0]:item[1]]
    return final_seq

In [ ]:
# exon to amino
def translate(seq,seq_dict): 
    table = {
        'ATA':'I', 'ATC':'I', 'ATT':'I', 'ATG':'M',
        'ACA':'T', 'ACC':'T', 'ACG':'T', 'ACT':'T',
        'AAC':'N', 'AAT':'N', 'AAA':'K', 'AAG':'K',
        'AGC':'S', 'AGT':'S', 'AGA':'R', 'AGG':'R',                
        'CTA':'L', 'CTC':'L', 'CTG':'L', 'CTT':'L',
        'CCA':'P', 'CCC':'P', 'CCG':'P', 'CCT':'P',
        'CAC':'H', 'CAT':'H', 'CAA':'Q', 'CAG':'Q',
        'CGA':'R', 'CGC':'R', 'CGG':'R', 'CGT':'R',
        'GTA':'V', 'GTC':'V', 'GTG':'V', 'GTT':'V',
        'GCA':'A', 'GCC':'A', 'GCG':'A', 'GCT':'A',
        'GAC':'D', 'GAT':'D', 'GAA':'E', 'GAG':'E',
        'GGA':'G', 'GGC':'G', 'GGG':'G', 'GGT':'G',
        'TCA':'S', 'TCC':'S', 'TCG':'S', 'TCT':'S',
        'TTC':'F', 'TTT':'F', 'TTA':'L', 'TTG':'L',
        'TAC':'Y', 'TAT':'Y', 'TAA':'_', 'TAG':'_',
        'TGC':'C', 'TGT':'C', 'TGA':'_', 'TGG':'W',
    }
    loc=seq.find("ATG")
    seq=seq[loc:]
    protein =""
    while (len(protein)!=len(seq_dict["amino"]) and ("ATG" in seq)):
        protein=""
        for i in range(0, len(seq), 3):
            codon = seq[i:i + 3]
            if table[codon]!="_":
                protein+= table[codon]
            else:
                break
        if len(protein)!=len(seq_dict["amino"]):
            seq=seq[1:]
            if "ATG" in seq:
                loc=seq.find("ATG")
                seq=seq[loc:]
            else:
                break
        else:
            break
    return protein

In [ ]:
# integrated function
def workflow(filename,transcript_table,seq_dict,id):
    table=table_info(filename)
    wt_seq=seq_dict["genomic"]
    flag=anti(table)
    mutant_seq=mutants(transcript_table,id,wt_seq,flag)
    #print("mutant:" + str(mutant_seq==seq_dict["genomic"]) )
    exon=exons(table,mutant_seq)
    #print("Exon:" + str(exon==seq_dict["exon"]))
    protein=translate(exon,seq_dict)
    #print("Protein: "+ str(protein==seq_dict["amino"]))
    return protein

In [ ]:
protein_list={}
for item in all_sequence.keys():
    filename="data/"+item+"/table.csv"
    seq_dict=all_sequence[item]
    id=transcript_ids[item]
    protein_list[item]=workflow(filename,data,seq_dict,id)

Mismatch between reference and ensembl: 0
Mismatch between reference and ensembl: 0
Mismatch between reference and ensembl: 0
Mismatch between reference and ensembl: 0
Mismatch between reference and ensembl: 0


In [ ]:
protein_list

{'9': 'MDPPSLDTAIQHALAGLYPPFEATAPTVLGQVFRLLDSGFQGDGLSFLLDFLIPAKRLCEQVREAACAPYSHCLFLHEGWPLCLRDEVVVHLAPLNPLLLRQGDFYLQVEPQEEQSVCIMIKCLSLDLCTVDKKPVPEPAYPILFTQEWLEAINSDFEGNPLHNCLVASENGIAPVPWTKITSPEFVDDRPQVVNALCQAWGPLPLEALDLSSPQELHQASSPDNQVLPAQSLAKGKGRTYGSKYPGLIKVEQARCGEVAFRMDEVVSQDFEGDYVALLGFSQESRGESPSREAGTSSGCTSGALEEIAGTKETPLFQKILPLSEANEGPSLGNRACTNPESSEERPYNLGFRRKVNLKAPTHNSERPPQGSYMNVLEDALDCASGLRAGVSQEPAASKMQGPLGNPENMVQLRPGPRQASSPRLSPASPAAAASETKIEVKTKERNGRLPKPMPCPSRNTSSPEPPTPGLKFSFLRGQRQPSVTPEKASLQHNGPWKVLCSLYSPKPNRAKSLGKAGTTQTKTSGPATAPSPLTEEKAALPEASAGSPERGPTLEEEPPGPEPRIGALGVKVFRSRIACLPGGRDRAGRPLLLVSTTEGAWEAPWCTVSEVTKLLSYLCTIPRPEDKAKGLAVLIDARRQPPQPGLVSALQATQAQVPASIRAILFLGEKEAALQLQTLPDVQVEVLTSLKALSHHVDPSQLPAVLEGPFPYCHTEWVHFFQKLDPFLADLHQASSLLQASIEEFEKADPPGGMQEATRCLSKSKELMEAVLRDPGLLGLQREGGATLARLQHDASRLDFSPDVRSHLAAATALYSLVDEQLHVLVTASNSLLGKLELRVRLGRLEAAIHQVSDWMEQEGRRCLQSLTPKDGSLETVEKAHAEFENFFLQAAAQYRRGLELSKQAAQLGATARGAGEAERAEFPELAAFASTQRAFQAKLTHFYMAAERQRTDLETLLHLHRFCKRMTWFHMDCQDLMAQLRLDKTSRVSPG

In [ ]:
for item in protein_list.keys():
    print(len(protein_list[item])==len(all_sequence[item]["pdb"]))

True
True
True
True
True


In [ ]:
for item in protein_list.keys():
    filename="data/"+item+"/mutant_seq.txt"
    with open(filename, 'w') as f:
        f.write(protein_list[item])

### 2.4 How is mutant sequence different from WT sequence?
* Mutant sequence is compared to the WT sequence to see how many non-synoymous mutation exists
* Mutant sequence which contains non-synonymous mutations are saved and its structure is predicted using the alphafold 2 colab (https://colab.research.google.com/github/sokrypton/ColabFold/blob/main/AlphaFold2.ipynb)

In [ ]:
def dif_seq(seq1,seq2):
    seq1=np.array(list(seq1))
    seq2=np.array(list(seq2))
    print("Matches: "+str(np.sum(seq1==seq2))+"/"+str(seq1.shape[0]))

In [ ]:
for item in protein_list.keys():
    print(item)
    seq1=protein_list[item]
    seq2=all_sequence[item]["amino"]
    dif_seq(seq1,seq2)

9
Matches: 1197/1200
8
Matches: 696/696
4
Matches: 347/348
3
Matches: 361/364
2
Matches: 406/406


## 3. Structure Analysis
* With the predicted mutant structure, LDDT is calculated to see how mutation alters the structure. Since LDDT requires exact match between the model and reference sequence. Mutation residues are removed. Calculation is performed on rank 1 sequence and wt
* Website for LDDT: https://swissmodel.expasy.org/lddt

In [ ]:
def find_mismatch(seq1,seq2):
    seq1=np.array(list(seq1))
    seq2=np.array(list(seq2))
    flag=(seq1==seq2)
    print(np.where(flag==0))

In [ ]:
for item in protein_list.keys():
    print(item)
    seq1=protein_list[item]
    seq2=all_sequence[item]["amino"]
    find_mismatch(seq1,seq2)

9
(array([ 338,  939, 1044]),)
8
(array([], dtype=int64),)
4
(array([260]),)
3
(array([266, 277, 302]),)
2
(array([], dtype=int64),)


### Trash

### 2.1 Extract wt sequence from fasta

In [ ]:
def table_info(filename):
    data=pd.read_csv(filename,thousands=',').iloc[1:-1,:]
    data[["Start","End","Length"]]=data[["Start","End","Length"]].astype("int")
    data=data[data["Exon / Intron"].apply(lambda x: x.startswith("E"))]
    return data

def anti(table):
    if (table["Start"].to_list()[0])>(table["End"].to_list()[0]):
        return True
    else:
        return False
    
def compliment(sequence):
    comp_dict = {
        "A":"T",
        "T":"A",
        "G":"C",
        "C":"G",
    }
    sequence_list = list(sequence)
    sequence_list = [comp_dict[base] for base in sequence_list]
    string = ''.join(sequence_list)
    return string

def mutants(transcript_table,id,wt_seq,anti):
    transcript_table=transcript_table[transcript_table["transcript_id"].apply(lambda x: x.startswith(id))]
    if anti:
        wt_seq=compliment(wt_seq)
        wt_seq=list(wt_seq.strip())
        loc=(transcript_table["V3.x"]-transcript_table["transcript_start"]).to_list()
        n=0
        for i in range(len(loc)):
            item=loc[i]
            if (wt_seq[-item]==transcript_table["V6.x"].to_list()[i])==False:
                n+=1
            #else:
               # wt_seq[-item]=transcript_table["V7.x"].to_list()[i] 
        print("Mismatch between reference and ensembl: "+str(n))
        wt_seq="".join(wt_seq)
        wt_seq=compliment(wt_seq)
    else:
        loc=(transcript_table["V2.x"]-transcript_table["transcript_start"]).to_list()
        wt_seq=list(wt_seq.strip())
        n=0
        for i in range(len(loc)):
            item=loc[i]
            if (wt_seq[item]==transcript_table["V6.x"].to_list()[i])==False:
                n+=1
           # else:
             #   wt_seq[item]=transcript_table["V7.x"].to_list()[i]
        print("Mismatch between reference and ensembl: "+str(n))
        wt_seq="".join(wt_seq)
    return wt_seq

def exons(table,wt_seq):
    relative_start=[]
    start=table["Start"].to_list()[0]
    for item in table["Start"].to_list():
        relative_start.append(np.abs(item-start))
    relative_end=[]
    for item in table["End"].to_list():
        relative_end.append(np.abs(item-start)+1)
    final_seq=""
    for item in zip(relative_start,relative_end):
        final_seq+=wt_seq[item[0]:item[1]]
    return final_seq

def translate(seq,seq_dict): 
    table = {
        'ATA':'I', 'ATC':'I', 'ATT':'I', 'ATG':'M',
        'ACA':'T', 'ACC':'T', 'ACG':'T', 'ACT':'T',
        'AAC':'N', 'AAT':'N', 'AAA':'K', 'AAG':'K',
        'AGC':'S', 'AGT':'S', 'AGA':'R', 'AGG':'R',                
        'CTA':'L', 'CTC':'L', 'CTG':'L', 'CTT':'L',
        'CCA':'P', 'CCC':'P', 'CCG':'P', 'CCT':'P',
        'CAC':'H', 'CAT':'H', 'CAA':'Q', 'CAG':'Q',
        'CGA':'R', 'CGC':'R', 'CGG':'R', 'CGT':'R',
        'GTA':'V', 'GTC':'V', 'GTG':'V', 'GTT':'V',
        'GCA':'A', 'GCC':'A', 'GCG':'A', 'GCT':'A',
        'GAC':'D', 'GAT':'D', 'GAA':'E', 'GAG':'E',
        'GGA':'G', 'GGC':'G', 'GGG':'G', 'GGT':'G',
        'TCA':'S', 'TCC':'S', 'TCG':'S', 'TCT':'S',
        'TTC':'F', 'TTT':'F', 'TTA':'L', 'TTG':'L',
        'TAC':'Y', 'TAT':'Y', 'TAA':'_', 'TAG':'_',
        'TGC':'C', 'TGT':'C', 'TGA':'_', 'TGG':'W',
    }
    loc=seq.find("ATG")
    seq=seq[loc:]
    protein =""
    while (len(protein)!=len(seq_dict["amino"]) and ("ATG" in seq)):
        protein=""
        for i in range(0, len(seq), 3):
            codon = seq[i:i + 3]
            if table[codon]!="_":
                protein+= table[codon]
            else:
                break
        if len(protein)!=len(seq_dict["amino"]):
            seq=seq[1:]
            if "ATG" in seq:
                loc=seq.find("ATG")
                seq=seq[loc:]
            else:
                break
        else:
            break
    return protein

def workflow(filename,transcript_table,seq_dict,id):
    table=table_info(filename)
    wt_seq=seq_dict["genomic"]
    flag=anti(table)
    mutant_seq=mutants(transcript_table,id,wt_seq,flag)
    print("mutant:" + str(mutant_seq==seq_dict["genomic"]) )
    exon=exons(table,mutant_seq)
    print("Exon:" + str(exon==seq_dict["exon_only"]))
    protein=translate(exon,seq_dict)
    print("Protein: "+ str(protein==seq_dict["amino"]))
    return protein

In [ ]:
def anti(table):
    if (table["Start"].to_list()[0])>(table["End"].to_list()[0]):
        return True
    else:
        return False

In [ ]:
def compliment(sequence):
    comp_dict = {
        "A":"T",
        "T":"A",
        "G":"C",
        "C":"G",
    }
    sequence_list = list(sequence)
    sequence_list = [comp_dict[base] for base in sequence_list]
    string = ''.join(sequence_list)
    return string

In [ ]:
def mutants(transcript_table,id,wt_seq,anti):
    transcript_table=transcript_table[transcript_table["transcript_id"].apply(lambda x: x.startswith(id))]
    if anti:
        wt_seq=compliment(wt_seq)
        wt_seq=list(wt_seq.strip())
        loc=(transcript_table["V3.x"]-transcript_table["transcript_start"]).to_list()
        n=0
        for i in range(len(loc)):
            item=loc[i]
            if (wt_seq[-item]==transcript_table["V6.x"].to_list()[i])==False:
                n+=1
            #else:
               # wt_seq[-item]=transcript_table["V7.x"].to_list()[i] 
        print("Mismatch between reference and ensembl: "+str(n))
        wt_seq="".join(wt_seq)
        wt_seq=compliment(wt_seq)
    else:
        loc=(transcript_table["V2.x"]-transcript_table["transcript_start"]).to_list()
        wt_seq=list(wt_seq.strip())
        n=0
        for i in range(len(loc)):
            item=loc[i]
            if (wt_seq[item]==transcript_table["V6.x"].to_list()[i])==False:
                n+=1
           # else:
             #   wt_seq[item]=transcript_table["V7.x"].to_list()[i]
        print("Mismatch between reference and ensembl: "+str(n))
        wt_seq="".join(wt_seq)
    return wt_seq

In [ ]:
# def mutants(transcript_table,id,wt_seq):
# #introduce mutation
#     transcript_table=transcript_table[transcript_table["transcript_id"].apply(lambda x: x.startswith(id))]
#     loc=(transcript_table["V2.x"]-transcript_table["transcript_start"]).to_list()
#     wt_seq=list(wt_seq.strip())
#     n=0
#     for i in range(len(loc)):
#         item=loc[i]
#         if (wt_seq[item]==transcript_table["V6.x"].to_list()[i])==False:
#             n+=1
#        # else:
#          #   wt_seq[item]=transcript_table["V7.x"].to_list()[i]
#     print("Mismatch between reference and ensembl: "+str(n))
#     return "".join(wt_seq)

In [ ]:
def exons(table,wt_seq):
    relative_start=[]
    start=table["Start"].to_list()[0]
    for item in table["Start"].to_list():
        relative_start.append(np.abs(item-start))
    relative_end=[]
    for item in table["End"].to_list():
        relative_end.append(np.abs(item-start)+1)
    final_seq=""
    for item in zip(relative_start,relative_end):
        final_seq+=wt_seq[item[0]:item[1]]
    return final_seq

In [ ]:
def translate(seq,seq_dict): 
    table = {
        'ATA':'I', 'ATC':'I', 'ATT':'I', 'ATG':'M',
        'ACA':'T', 'ACC':'T', 'ACG':'T', 'ACT':'T',
        'AAC':'N', 'AAT':'N', 'AAA':'K', 'AAG':'K',
        'AGC':'S', 'AGT':'S', 'AGA':'R', 'AGG':'R',                
        'CTA':'L', 'CTC':'L', 'CTG':'L', 'CTT':'L',
        'CCA':'P', 'CCC':'P', 'CCG':'P', 'CCT':'P',
        'CAC':'H', 'CAT':'H', 'CAA':'Q', 'CAG':'Q',
        'CGA':'R', 'CGC':'R', 'CGG':'R', 'CGT':'R',
        'GTA':'V', 'GTC':'V', 'GTG':'V', 'GTT':'V',
        'GCA':'A', 'GCC':'A', 'GCG':'A', 'GCT':'A',
        'GAC':'D', 'GAT':'D', 'GAA':'E', 'GAG':'E',
        'GGA':'G', 'GGC':'G', 'GGG':'G', 'GGT':'G',
        'TCA':'S', 'TCC':'S', 'TCG':'S', 'TCT':'S',
        'TTC':'F', 'TTT':'F', 'TTA':'L', 'TTG':'L',
        'TAC':'Y', 'TAT':'Y', 'TAA':'_', 'TAG':'_',
        'TGC':'C', 'TGT':'C', 'TGA':'_', 'TGG':'W',
    }
    loc=seq.find("ATG")
    seq=seq[loc:]
    protein =""
    while (len(protein)!=len(seq_dict["amino"]) and ("ATG" in seq)):
        protein=""
        for i in range(0, len(seq), 3):
            codon = seq[i:i + 3]
            if table[codon]!="_":
                protein+= table[codon]
            else:
                break
        if len(protein)!=len(seq_dict["amino"]):
            seq=seq[1:]
            if "ATG" in seq:
                loc=seq.find("ATG")
                seq=seq[loc:]
            else:
                break
        else:
            break
    return protein

In [ ]:
def workflow(filename,transcript_table,seq_dict,id):
    table=table_info(filename)
    wt_seq=seq_dict["genomic"]
    flag=anti(table)
    mutant_seq=mutants(transcript_table,id,wt_seq,flag)
    print("mutant:" + str(mutant_seq==seq_dict["genomic"]) )
    exon=exons(table,mutant_seq)
    print("Exon:" + str(exon==seq_dict["exon_only"]))
    protein=translate(exon,seq_dict)
    print("Protein: "+ str(protein==seq_dict["amino"]))
    return protein,exon